In [1]:
from sklearn.datasets import fetch_covtype
'''
covtype
대표 수종(나무의 종류) 데이터
미국 삼림을 30*30m 영역으로 나누어 각 영역으로부터 대표 수정을 예측하기 위한 데이터
수종 7종류, 특징 54종류, 표본데이터 58만개
'''

'\nfetch_covtype\n대표 수종(나무의 종류) 데이터\n미국 삼림을 30*30m 영역으로 나누어 각 영역으로부터 대표 수정을 예측하기 위한 데이터\n수종 7종류, 특징 54종류, 표본데이터 58만개\n'

In [2]:
covtype = fetch_covtype(shuffle=True, random_state=0)
data = covtype.data

In [4]:
target = covtype.target-1

In [5]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


In [6]:
np.unique(target)

array([0, 1, 2, 3, 4, 5, 6])

In [7]:
x_train, x_test, y_train, y_test = train_test_split(data, target)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((435759, 54), (145253, 54), (435759,), (145253,))

In [8]:
# minmax scaling
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [12]:
# 특정범위만큼 데이터를 읽어오는 함수
# 파일, 또는 DB에서 읽어오는 함수
def read_data(start, end):
    idx = list(range(start,min(end, len(x_train)-1)))
    x = x_train[idx,:]
    y = y_train[idx]
    return x, y

### SGD
* 퍼셉트론 모형에서 가중치를 업데이트해가는 방식

In [20]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score

model = SGDClassifier(random_state=0)
n = 10
n_x = len(x_train) // n
epochs = 10
for epoch in range(epochs):
    for nn in range(n):
        x, y = read_data(n_x*nn, n_x*(nn+1))
        model.partial_fit(x,y,classes=np.unique(target))
    acc = accuracy_score(y_test, model.predict(x_test))
    print('EPOCH : {}, ACC : {}'.format(epoch, acc))

EPOCH : 0, ACC : 0.7097271657039785
EPOCH : 1, ACC : 0.7107047702973432
EPOCH : 2, ACC : 0.7108149229275815
EPOCH : 3, ACC : 0.7100920462916429
EPOCH : 4, ACC : 0.7097615884009281
EPOCH : 5, ACC : 0.7097891265584876
EPOCH : 6, ACC : 0.7101884298431014
EPOCH : 7, ACC : 0.7098442028736067
EPOCH : 8, ACC : 0.7102159680006609
EPOCH : 9, ACC : 0.710085161752253


### NaiveBayes
* 일부데이터를 이용해서 사전확률분포로 사용

In [23]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score

model = BernoulliNB()
n = 10
n_x = len(x_train) // n

for nn in range(n):
    x, y = read_data(n_x*nn, n_x*(nn+1))
    model.partial_fit(x,y,classes=np.unique(target))
    acc = accuracy_score(y_test, model.predict(x_test))
    print('EPOCH : {}, ACC : {}'.format(nn, acc))

EPOCH : 0, ACC : 0.6309129587684936
EPOCH : 1, ACC : 0.6344653810936779
EPOCH : 2, ACC : 0.6325790173008475
EPOCH : 3, ACC : 0.6325032873675587
EPOCH : 4, ACC : 0.6293914755633274
EPOCH : 5, ACC : 0.6324482110524395
EPOCH : 6, ACC : 0.6331710876883782
EPOCH : 7, ACC : 0.6324619801312193
EPOCH : 8, ACC : 0.6329645515066814
EPOCH : 9, ACC : 0.6329783205854612


In [ ]:
# 앙상블 모델에서 각 모델간의 다른 데이터를 사용해서 학습 가능